<a href="https://colab.research.google.com/github/maloyan/HW_keras/blob/master/hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#DON'T FORGET TO CHANGE Runtime->GPU

In [94]:
!nvidia-smi

Tue Jun  2 20:49:51 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P0    59W / 149W |   2409MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [95]:
!wget https://data.mendeley.com/datasets/rscbjbr9sj/2/files/f12eaf6d-6023-432f-acc9-80c9d7393433/ChestXRay2017.zip

--2020-06-02 20:49:53--  https://data.mendeley.com/datasets/rscbjbr9sj/2/files/f12eaf6d-6023-432f-acc9-80c9d7393433/ChestXRay2017.zip
Resolving data.mendeley.com (data.mendeley.com)... 162.159.133.86, 162.159.130.86, 2606:4700:7::a29f:8556, ...
Connecting to data.mendeley.com (data.mendeley.com)|162.159.133.86|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com/31ab5ede-ed34-46d4-b1bf-c63d70411497 [following]
--2020-06-02 20:49:54--  https://md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com/31ab5ede-ed34-46d4-b1bf-c63d70411497
Resolving md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com (md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com)... 52.218.37.136
Connecting to md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com (md-datasets-public-files-prod.s3.eu-west-1.amazonaws.com)|52.218.37.136|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1235512

In [0]:
!unzip ChestXRay2017.zip

Archive:  ChestXRay2017.zip
replace chest_xray/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras import backend as K

import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
# Строим модель
# Она состоит из трех блоков вида: Свертка->Активация->MaxPooling

# После мы получаем признаки, которые запишем в одномерный массив с помощью Flatten
# Полносвязный слой выход 64->
# Активация Relu->
# 50% Дропаут ->
# Полносвязный слой с одним выходом->
# Нормализация с помощью Sigmoid

def build_model(input_shape):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(64))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    return model

In [0]:
#Посмотрим на пример из обучающей и тестовой выборки

In [0]:
img_normal = load_img('./chest_xray/train/NORMAL/NORMAL2-IM-0588-0001.jpeg')

print('NORMAL')
plt.imshow(img_normal)
plt.show()

In [0]:
img_pneumonia = load_img('./chest_xray/train/PNEUMONIA/person63_bacteria_306.jpeg')

print('PNEUMONIA')
plt.imshow(img_pneumonia)
plt.show()

In [0]:
# Составим Датафрейм с путями и лейблами
# Он нам понадобится для Dataloader 
train = []
train_y = []
test = []
test_y = []
for i in ['PNEUMONIA', 'NORMAL']:
    for j in os.listdir('./chest_xray/train/' + i + '/'):
        train.append(i + '/' + j)
        train_y.append(i)

    for j in os.listdir('./chest_xray/test/' + i + '/'):
        test.append(i + '/' + j)
        test_y.append(i)

df_train = pd.DataFrame({'dir':train, 'label':train_y})
df_test = pd.DataFrame({'dir':test, 'label':test_y})

In [0]:
df_train.head()

In [0]:
IMG_WIDTH, IMG_HEIGHT = 224, 224

TRAIN_VAL_SPLIT = 0.8
EPOCHS = 10
BATCH = 16

TRAIN_SAMPLES = df_train.shape[0] * TRAIN_VAL_SPLIT
VAL_SAMPLES = df_train.shape[0] * (1 - TRAIN_VAL_SPLIT)

TRAIN_STEPS = TRAIN_SAMPLES // BATCH
VAL_STEPS = VAL_SAMPLES // BATCH

if K.image_data_format() == 'channels_first':
    INPUT_SHAPE = (3, IMG_WIDTH, IMG_HEIGHT)
else:
    INPUT_SHAPE = (IMG_WIDTH, IMG_HEIGHT, 3)

In [0]:
model = build_model(INPUT_SHAPE)
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [0]:
# this is the augmentation configuration we will use for training
datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    validation_split=0.2,
    horizontal_flip=True)

In [0]:
# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [0]:
train_generator=datagen.flow_from_dataframe(
    dataframe=df_train,
    directory="./chest_xray/train/",
    x_col="dir",
    y_col="label",
    subset="training",
    batch_size=BATCH,
    seed=42,
    shuffle=True,
    class_mode="binary",
    target_size=(IMG_WIDTH, IMG_HEIGHT))

In [0]:

validation_generator=datagen.flow_from_dataframe(
    dataframe=df_train,
    directory="./chest_xray/train/",
    x_col="dir",
    y_col="label",
    subset="validation",
    batch_size=BATCH,
    seed=42,
    shuffle=True,
    class_mode="binary",
    target_size=(IMG_WIDTH, IMG_HEIGHT))

In [0]:
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH,
    class_mode='binary')

In [0]:
model.fit_generator(
    train_generator,
    steps_per_epoch=TRAIN_STEPS,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=VAL_STEPS)